In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from stix2.v21 import (ThreatActor, Identity, AttackPattern, Campaign, IntrusionSet, Relationship, ExternalReference, Bundle, Grouping)
from efficient_apriori import apriori
import re
from pyattck import Attck
import requests
from stix2 import MemoryStore, Filter

In [3]:
# Settings Values - for apriori algorithm
confidenceLevel = 0.70
supportLevel = 0.05
abstract = False

In [4]:
def GenerateAprioriLists():
    # some data comes from this dataset with TTPs
    df = pd.read_csv("datasets/Categorized_Adversary_TTPs.csv") # sample dataset of attacks


    # more data is gained by using attck data from tool and malware TTPs
    attack = Attck()

    malwares = attack.enterprise.malwares + attack.enterprise.tools
    ttpLists = []
    for malware in malwares:
        ttpLists.append(["'" + ttp.id + "'" for ttp in malware.techniques])

    # to use the apriori we need to generate a list of lists
    aprList = ttpLists
    for row in df.values:
        aprList.append((row[13].strip('][').split(', ')))
    return aprList

In [5]:
def AbstractTTPs(ttpList):
    # take sub-techniques and remove the .### to abstract them to parent techniques 
    for i in range(0,len(ttpList)):
        ttpList[i] = [re.sub(r'\.[0-9]+', '', ttp) for ttp in ttpList[i]]
    return ttpList 

In [6]:
# takes a list of lists and returns a list of rules sorted by size 
def AprioriMining(aprList):
    # perform apriori rule association mining
    itemsets, rules = apriori(aprList, min_support=supportLevel, min_confidence=confidenceLevel)
    
    # sort by size to get the 1:1 mappings first and so on. 
    ruleNums = np.array([len(rule.lhs+rule.rhs) for rule in rules])
    rules = np.array(rules)
    inds = ruleNums.argsort()[::]
    rules = rules[inds]
    
    # maximum rule size of 4 to limit number of rules, any rules with size > 4 are redundant anyways
    rules = [x for x in filter(lambda rule: len(rule.lhs+rule.rhs) <= 4, rules)]
    return rules

In [7]:
# downloads latest MITRE framework from the branch
def get_data_from_branch(domain):
    """get the ATT&CK STIX data from MITRE/CTI. Domain should be 'enterprise-attack', 'mobile-attack' or 'ics-attack'. Branch should typically be master."""
    stix_json = requests.get(f"https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/{domain}/{domain}.json").json()
    return MemoryStore(stix_data=stix_json["objects"])

src = get_data_from_branch("enterprise-attack")

In [8]:
# get TTPs from seed, where seed is an APT group code: GXXXX
def ExtractSeedTTPs(seed):
    seeds = []
    actor = actors[seed]
    for ttp in actor.techniques:
        seeds.append(ttp.id)
        
    # if we are using abstracted TTPs then remove sub technique
    if (abstract):
        for i in range(0, len(seeds)):
            seeds[i] = re.sub(r'\.[0-9]+', '', seeds[i])
    return seeds

In [9]:
def ExportBundle(bundle, filename):
    with open(filename, "w") as f:
        f.write(bundle.serialize())
        f.close()

In [10]:
# each rule has a confidence, lift, support, lhs, rhs, conviction, rule power factor (RPF), 

# takes an APTGroup and generates hypothesized relationships and attack pattern objects
# returns a tuple of: (displayInfo, attackPatterns, relationships, groupings, mitigationObjects)
def CreateRelationships(seeds, aprioriLists):
    displayInfo = [] #stores the TTP and tactic for easy logging
    seen = [] #stores TTPs that have been seen by the algorithm
    attackPatterns = [] #stores generated attack pattern objects
    relationships = {} # stores generated relationship objects
    groupings = {} # stores generated grouping objects

    
    
    # get seeds out of the APT group
    #seeds = ExtractSeedTTPs(APTGroup)
    seeds = ["T1566", "T1204"]
    
    # perform association rule mining
    rules = AprioriMining(aprioriLists) 
    
    #add seeds to activity-attack-graph as nodes
    for seed in seeds:
        # get the stix object for the TTP
        ttp = src.query([ Filter("external_references.external_id", "=", seed) ])[0]
        
        # create and add objects to lists, context is used to identify seeds from hypothesized events
        groupings[seed] = (Grouping(object_refs=[ttp.id], context = "Seed Event"))
        attackPatterns.append(src.query([ Filter("external_references.external_id", "=", seed) ])[0])
    
    seedTotals = ["'"+x+"'" for x in seeds]
    # use a queue to iterate through and create a tree of TTPs
    while len(seeds) > 0:
        for rule in rules:
            # check to see if the left hand side of a rule is satisfied 
            if "'"+seeds[0]+"'" in rule.lhs and set(rule.lhs).issubset(seedTotals):
                #if the lhs is satisfied, then loop through each TTP in the rhs 
                for ttpName in rule.rhs:
                    # if this TTP hasn't been visited already then create STIX objects
                    if ttpName not in seen:
                        ttp = src.query([ Filter("external_references.external_id", "=", ttpName[1:-1]) ])[0]
                        
                        # track rule tactics for logging to displayInfo
                        tactics = []
                        for i in ttp['kill_chain_phases']:
                            tactics.append(i['phase_name'])
                        
                        # create a new grouping object with the rule name. 
                        groupings[ttpName[1:-1]] = (Grouping(object_refs=[ttp.id], context = "Hypothesized Event"))
                        
                        

                        
                        displayInfo.append([ttp['name'], ttpName[1:-1], tactics])
                        attackPatterns.append(ttp)
                        seedTotals.append(ttpName)
                        seen.append(ttpName)
                        seeds.append(ttp['external_references'][0]['external_id'])

                    # if the relationship already exists between two objects then we take the one with higher confidence
                    # make sure the exact relationship does not alraedy exist
                    if not (groupings[seeds[0]]['id'], groupings[ttpName[1:-1]]['id']) in relationships:
                        # if the opposite relationship exists then choose the one with the highest confidence to add
                        # we do this because if we do not then on the graph there will be two arrows going opposite directions to connect the same 2 TTPs
                        # this causes clutter and makes the confidence values unreadable since they will be layered on top of each other
                        if (groupings[ttpName[1:-1]]['id'], groupings[seeds[0]]['id']) in relationships:
                            # if existing relationship has higher confidence, move on
                            if float(relationships[(groupings[ttpName[1:-1]]['id'], groupings[seeds[0]]['id'])]['relationship_type']) > rule.confidence:
                                continue # move to next iteration of for loop, so code after this statement won't execute
                            # if existing relationship has lower confidence, delete it and let the new relationship take it's place
                            else:
                                del relationships[(groupings[ttpName[1:-1]]['id'], groupings[seeds[0]]['id'])]
                        relationships[(groupings[seeds[0]]['id'], groupings[ttpName[1:-1]]['id'])] = Relationship(groupings[seeds[0]]['id'], str(round(rule.confidence, 3)), groupings[ttpName[1:-1]]['id'])
        
        # pop to progress the queue
        seeds.pop(0)

    return (displayInfo, attackPatterns, relationships, groupings)

In [11]:
aprioriList = AbstractTTPs(GenerateAprioriLists()) # generate lists for apriori

In [12]:
intelSeed = ["T1566", "T1204"]
displayInfo, attackPatterns, relationships, groupings = CreateRelationships(intelSeed, aprioriList)

In [13]:
# Displays the results of the Apiori Algorithm 
for info in displayInfo:
    print(f"{info[1]}:{info[0]}{info[2]}")
    print("")

T1059:Command and Scripting Interpreter['execution']

T1105:Ingress Tool Transfer['command-and-control']

T1204:User Execution['execution']

T1027:Obfuscated Files or Information['defense-evasion']

T1071:Application Layer Protocol['command-and-control']

T1082:System Information Discovery['discovery']

T1547:Boot or Logon Autostart Execution['persistence', 'privilege-escalation']

T1566:Phishing['initial-access']

T1140:Deobfuscate/Decode Files or Information['defense-evasion']

T1057:Process Discovery['discovery']

T1083:File and Directory Discovery['discovery']

T1070:Indicator Removal['defense-evasion']



In [14]:
# Bundle STIX Objects for Visualization
bundle = Bundle(attackPatterns+list(groupings.values())+list(relationships.values()), allow_custom=True)

In [15]:
# Export bundle for visualization here: https://github.com/yukh1402/cti-stix-diamond-activity-attack-graph
ExportBundle(bundle, "test.json")

In [16]:
# Phase 2: Make APT Threat Profiles
attack = Attck()

# Dictionary to hold all the threat profiles. 
# The key is a given APT and the values are a list of TTPs
threat_profiles = {}

for actor in attack.enterprise.actors:
    technique_list = []
    for technique in actor.techniques:
        technique_list.append(technique.id)
    threat_profiles[actor.id] = technique_list
    
print(threat_profiles)

{'G0032': ['T1573.001', 'T1070.003', 'T1588.002', 'T1560.002', 'T1098', 'T1005', 'T1583.006', 'T1059.001', 'T1105', 'T1564.001', 'T1008', 'T1593.001', 'T1070', 'T1614.001', 'T1057', 'T1001.003', 'T1542.003', 'T1587.001', 'T1124', 'T1021.004', 'T1204.002', 'T1584.001', 'T1543.003', 'T1547.009', 'T1065', 'T1049', 'T1202', 'T1491.001', 'T1591.004', 'T1059.005', 'T1078', 'T1055.001', 'T1566.002', 'T1104', 'T1218.011', 'T1189', 'T1047', 'T1218.010', 'T1489', 'T1591', 'T1566.001', 'T1585.001', 'T1012', 'T1574.002', 'T1585.002', 'T1021.002', 'T1608.001', 'T1561.002', 'T1016', 'T1090.002', 'T1534', 'T1553.002', 'T1090.001', 'T1140', 'T1036', 'T1620', 'T1485', 'T1547.001', 'T1071.001', 'T1218.005', 'T1589.002', 'T1087.002', 'T1033', 'T1497.001', 'T1110', 'T1221', 'T1134.002', 'T1588.004', 'T1204.001', 'T1059.003', 'T1048.003', 'T1027.007', 'T1583.004', 'T1102.002', 'T1588.003', 'T1132.001', 'T1529', 'T1560.003', 'T1027', 'T1557.001', 'T1036.005', 'T1026', 'T1027.002', 'T1562.001', 'T1574.013', 